In [1]:
%pip install scipy
%pip install numpy
%pip install matplotlib
%pip install pandas
%pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from scipy.stats import t
import numpy.matlib as mb
import numpy.linalg as lg
import ipywidgets as widgets
import math as mt
import numpy as np
import pandas as pd
import re

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def erroMedicao(VVC, xi):
  n = np.size(xi,0) # n - Tamanho das amostas
  mVVC = mb.repmat(VVC,n,1) # Replica o vetor coluna VVc em n linhas
  em = (xi - mVVC)
  emMaxAbs = np.max(abs(em),0).T
  return (emMaxAbs)

def erroSistematico(VVC, xi):
  vm = np.mean(xi,0) # Média aritmética de cada linha
  return ((vm - VVC).T) # Erro sistemático ou Tendência

def erroAleatorio(xi):
  n = np.std(xi,0,ddof=1)
  return (n.T)

def histerese(xiAvanco,xiRecuo):
  vmAvanco = np.mean(xiAvanco,0) # Média aritmética em avanço.
  vmRecuo = np.mean(xiRecuo,0) # Média aritmética em recuo | retorno.
  hist = abs(vmAvanco-vmRecuo)
  histMax = np.max(hist,0)
  return (np.asmatrix(hist).T)

def linearidade(VVC, xi):
  tam = np.size(VVC,1) # Quantidade de valores analisados
  VVC = VVC.T
  vm = np.mean(xi,0).T # Média aritmética de cada linha
  # Calcula a reta com menor erro em relação as medidas - MMQ
  mt = np.concatenate((vm, np.ones((tam,1))),1)
  x = lg.inv(mt.T @ mt)
  y = mt.T @ VVC
  coef = x @ y # Faz a linearização com os dados
  eVVC = coef[0,0]*vm+coef[1,0] # VVC estimado
  el = abs(VVC - eVVC)
  elMax = np.max(el, 0)
  return (el)

def repetitividade(xi):
  return(np.std(xi,0,ddof=1).T) # Calcula o Desvio Padrão

def incertezaPadraoTipoA(st, n): # Incerteza Padrão do Tipo A
  return (np.asmatrix(st/mt.sqrt(n)))

def incertezaPadraoTipoB(uTB,caracteristicasPadrao): #Incerteza Padrão do Tipo B
  desc = uTB[0]
  y = uTB[1]
  tipo = re.search(r"^([^ |\(])+",desc)
  if tipo!=None:
    desc = tipo.group(0)
  if desc == 'Res':
    resp = y/(2*mt.sqrt(3))
  elif tipo[0] == 'Deriva':
    resp = y/mt.sqrt(3)
  elif tipo[0] == 'Lin':
    resp = y/mt.sqrt(3)
    #resp = zeros(size(y,1),1)
  elif tipo[0] == 'Hist':
    resp = y/(2*mt.sqrt(3))
  else: # "Quando for Incerteza - U"
    try:
      resp = y/caracteristicasPadrao['k'] # fatorAbrang
    except:
      resp = y
  return (resp)

def incertezaCombinada(y): # Cálculo da Incerteza Combinada
  return(np.sqrt(np.sum(np.square(y), axis=1)))

def grauLiberdadeEfetivo(uc,uTA,n,uTB,caracteristicasPadrao): # CÁLCULO DO GRAU DE LIBERDADE EFETIVO
  numerador = np.power(uc,4)
  veff = mb.repmat((np.asmatrix(caracteristicasPadrao['veff'])).T,1,np.size(uTB,1))
  va = mb.repmat(np.asmatrix([n-1]),np.size(uTA,0),1) # Grau de Liberdade
  denominador = (np.power(uTA,4)/va)+np.sum(np.power(uTB,4)/veff, axis=1)
  vefF = numerador/denominador # GRAU DE LIBERDADE EFETIVO
  return(vefF)

def fatorAbrangencia(nivelConfianca,veff):
  k=t.ppf(1-((1-nivelConfianca/100)/2),veff)
  return(k)

def incertezaExpandida(uc,k):#Cálculo da Incerteza Expandida
  ue = np.multiply(uc,k);
  return (ue)

def caractEstatico(caracteristicasPadrao, uTB, nivelConfiancaDesejado, VVC, xi, xi_avanco, xi_atraso):
  em = erroMedicao(VVC, xi)
  es = erroSistematico(VVC, xi)
  ea = erroAleatorio(xi)
  rep = repetitividade(xi)
  lin = linearidade(VVC, xi)
  hist = histerese(xi_avanco,xi_atraso)

  uTA_padrao = incertezaPadraoTipoA(rep,np.size(xi,0))
  a = (np.asmatrix(np.array([incertezaPadraoTipoB(x,caracteristicasPadrao) for x in uTB]))).T
  b = incertezaPadraoTipoB(('Lin',lin),caracteristicasPadrao)
  c = incertezaPadraoTipoB(('Hist',hist),caracteristicasPadrao)
  uTB_padrao = np.concatenate((a, b, c), 1)
  uc = incertezaCombinada(np.concatenate((uTA_padrao, uTB_padrao),1))
  veff = grauLiberdadeEfetivo(uc,uTA_padrao,np.size(xi,0),uTB_padrao,caracteristicasPadrao)
  k = fatorAbrangencia((np.asmatrix(nivelConfiancaDesejado)).T,veff)
  ue = incertezaExpandida(uc,k)

  desc = ["EM","ES","EA","Rep","Hist","ELin","uTA (Rep_p)"]+[x[0]+"_p" for x in uTB]+["Lin_p",'Hist_p',"uC","Veff","k","uE"]
  data = np.concatenate((em, es, ea, rep, hist, lin, uTA_padrao, uTB_padrao, uc, veff, k, ue), 1)

  return(pd.DataFrame(data=data, columns=desc))

# Caracterização

In [7]:
caracteristicasPadrao = {'k': np.matrix([2, 2, 2, 2]),
                         'veff': np.matrix([np.Inf, np.Inf, np.Inf, np.Inf])}

uTB = [('Res',np.matrix([0.3, 0.3, 0.3, 0.3])),
       ('Res (p)',np.matrix([0.01, 0.01, 0.01, 0.01])),
       ('u (p)',np.matrix([0.001, 0.001, 0.001, 0.001])),
       ('Deriva (p)',np.matrix([0.002, 0.002, 0.002, 0.002]))]

nivelConfiancaDesejado = np.matrix([95.45,95.45,95.45,95.45])

VVC = np.matrix([5,60,120,180])

xi = np.matrix([[5.01, 58.9, 117.45, 184.5],
               [4.93, 58.82, 117.33, 183.12],
               [5.69, 58.24, 117.24, 180.51],
               [4.74, 58.31, 116.53, 181.98],
               [4.73, 58.17, 116.77, 180.1]])

caractEstatico(caracteristicasPadrao, uTB, nivelConfiancaDesejado, VVC, xi, np.ones((1,np.size(xi,1))), np.ones((1,np.size(xi,1))))

,EM,ES,EA,Rep,Hist,ELin,uTA (Rep_p),Res_p,Res (p)_p,u (p)_p,Deriva (p)_p,Lin_p,Hist_p,uC,Veff,k,uE
0,0.69,0.020,0.393573,0.393573,0.0,1.383474,0.176011,0.086603,0.002887,0.0005,0.001155,0.798749,0.0,0.822490,1907.304187,2.001314,1.646061
1,1.83,-1.512,0.344340,0.344340,0.0,0.627411,0.153994,0.086603,0.002887,0.0005,0.001155,0.362236,0.0,0.403037,187.684707,2.013410,0.811478
2,3.47,-2.936,0.394436,0.394436,0.0,2.576045,0.176397,0.086603,0.002887,0.0005,0.001155,1.487280,0.0,1.500209,20926.665600,2.000122,3.000602
3,4.50,2.042,1.825245,1.825245,0.0,1.819981,0.816274,0.086603,0.002887,0.0005,0.001155,1.050767,0.0,1.333388,28.480127,2.091686,2.789029
